In [391]:
import pandas as pd
import numpy as np

#Open the src\data\gadm41_COD_2.json
import json
import os

from luciferml.supervised.regression import Regression

In [400]:
# Get cwd
cwd = os.getcwd()
# cd .. to get to the parent directory
cwd = os.path.dirname(cwd)

In [588]:
# load the csv src\data\LifeExp\API_SP.DYN.LE00.IN_DS2_en_csv_v2_4770434.csv
df = pd.read_csv(os.path.join(cwd, 'data', 'LifeExp', 'API_SP.DYN.LE00.IN_DS2_en_csv_v2_4770434.csv'))
# Get the rows with Indicator Name = Life expectancy at birth, total (years)
df = df[df['Indicator Name'] == 'Life expectancy at birth, total (years)']
# Exclude the rows which are county aggregates
#df = df[df['Country Code'] != 'COD']
df.drop(['Country Name', 'Indicator Name', 'Indicator Code', 'Unnamed: 66', '2021'], axis=1, inplace=True)

# load the csv src\data\LifeExp\API_SP.DYN.LE00.IN_DS2_en_csv_v2_4770434.csv
c_codes = pd.read_csv(os.path.join(cwd, 'data', 'country_codes.csv'))
# Icodes = c_codes['alpha-3'].values
iso_codes = c_codes['alpha-3'].values

#Keep the rows with the iso codes in the iso_codes list
df = df[df['Country Code'].isin(iso_codes)]
df.set_index('Country Code', inplace=True)

# keep only data from 1998 to 2018
df = df.loc[:, '1999':'2006']

#Turn into a nested dictionary
df = df.to_dict(orient='index')
# Read into a dataframe
df = pd.DataFrame.from_dict(df, orient='index')

# Turn into a nested dataframe
df = df.stack().reset_index()
# Rename the columns to 'country' and 'year' and 'life_exp'
df.columns = ['country', 'year', 'life_exp']

nested_df_lex = df.groupby(['country', 'year']).last()

In [589]:
import os
population = pd.read_excel(os.path.join(cwd, 'data', 'pop_bancomundial_excel.xlsx'))
population = population[population['Indicator Name'] == 'Población, total']

# Set the column names to string
population.columns = population.columns.astype(str)

population.drop(['Country Name', 'Indicator Name', 'Indicator Code', '2021'], axis=1, inplace=True)

# load the csv src\data\LifeExp\API_SP.DYN.LE00.IN_DS2_en_csv_v2_4770434.csv
c_codes = pd.read_csv(os.path.join(cwd, 'data', 'country_codes.csv'))
# Icodes = c_codes['alpha-3'].values
iso_codes = c_codes['alpha-3'].values

population = population[population['Country Code'].isin(iso_codes)]
population.set_index('Country Code', inplace=True)

# keep only data from 1998 to 2018
population = population.loc[:, '1999':'2006']

#Turn into a nested dictionary
population = population.to_dict(orient='index')
# Read into a dataframe
population = pd.DataFrame.from_dict(population, orient='index')

# Turn into a nested dataframe
population = population.stack().reset_index()
# Rename the columns to 'country' and 'year' and 'life_exp'
population.columns = ['country', 'year', 'population']

nested_df_pop = population.groupby(['country', 'year']).last()

In [594]:
# LOAD THE csv src\data\aqli_global_data.csv
df = pd.read_csv(os.path.join(cwd, 'data', 'aqli_global_data.csv'))
# keep columns iso_alpha3, and columsn qith pm in the name
df = df[['iso_alpha3'] + [col for col in df.columns if 'pm' in col]]
# Remove the substring 'pm' from the column names
df.columns = [col.replace('pm_', '') for col in df.columns]
df = df[df['iso_alpha3'].isin(iso_codes)]
df.pivot_table(index='iso_alpha3', aggfunc='mean')

#Turn into a nested dictionary
df = df.to_dict(orient='index')
# Read into a dataframe
df = pd.DataFrame.from_dict(df, orient='index')

df.set_index('iso_alpha3', inplace=True)

#add "pm_" to the column names
#df.columns = ['pm_' + col for col in df.columns]

# Turn into a nested dataframe
df = df.stack().reset_index()

# Rename the columns to 'country' and 'year' and 'life_exp'
df.columns = ['country', 'year', 'pm']

nested_df_pm = df.groupby(['country', 'year']).last()

In [595]:
df

,iso_alpha3,level_1,0
0,ABW,1998,2.50
1,ABW,1999,3.61
2,ABW,2000,2.40
3,ABW,2001,2.48
4,ABW,2002,2.22
...,...,...,...
4951,ZWE,2014,14.38
4952,ZWE,2015,16.64
4953,ZWE,2016,14.90
4954,ZWE,2017,14.34


In [603]:
all_years = pd.DataFrame()
for a in range(1999,2007):
# Read the first page in teh xls file in src\data\finaal.xlsx into a dataframe
    all_years = pd.concat([all_years, pd.read_excel(os.path.join(cwd, 'data', 'finaal.xlsx'), sheet_name=str(a), header=0)])
#set the year column to the a string
all_years['year'] = all_years['year'].astype(str)
nested_df_pm_all = all_years.groupby(['country', 'year']).last()

In [604]:
nested_df_pop

population
country year            
ABW     1999    86867.00
        2000    89101.00
        2001    90691.00
        2002    91781.00
        2003    92701.00
...                  ...
ZWE     2002 11984644.00
        2003 12075828.00
        2004 12160881.00
        2005 12224753.00
        2006 12330490.00

[1720 rows x 1 columns]

In [600]:
nested_df_pm_all

min   max   avg
country year                  
AFG     1999 18.40 65.00 38.92
        2000 20.10 67.80 37.64
        2001 10.40 90.00 45.62
        2002 10.00 85.80 46.91
        2003 15.50 89.20 47.35
...            ...   ...   ...
ZWE     2002  7.30 13.90  9.86
        2003  6.90 12.00  8.72
        2004  6.20 10.70  7.64
        2005  7.20 13.20  8.97
        2006  6.60 11.80  8.15

[1512 rows x 3 columns]

In [605]:
#merge nested_df_lex and nested_df_pop
nested_df = pd.merge(nested_df_lex, nested_df_pop, on=['country', 'year'], how='inner')
nested_df = pd.merge(nested_df, nested_df_pm_all, on=['country', 'year'], how='inner')

In [608]:
nested_df.reset_index(inplace=True)

In [609]:
# Create the folder if it does not exist
if not os.path.exists(os.path.join(cwd, 'data', 'images')):
    os.makedirs(os.path.join(cwd, 'data', 'images'))

min_shape = 20
county_fake = []

all_df = pd.DataFrame()
#subset nested_df into dataframes for each country
for country in nested_df['country'].unique():
    nested_df_country = nested_df[nested_df['country'] == country]
    if len(nested_df_country) < 15:
        county_fake.append(country)
        min_shape = len(nested_df_country)

for country in nested_df['country'].unique():
    nested_df_country = nested_df[nested_df['country'] == country]
    #drop the column country
    if country in county_fake:
        continue
    else:
        #nested_df_country.drop('country', axis=1, inplace=True)
        #nested_df_country.tail(15).to_csv(os.path.join(cwd, 'data', 'images', str(country) + '.csv') , index=False)
        all_df = pd.concat([all_df, nested_df_country.tail(15)])

country_cluster_SOM = ['COD', 'COL', 'ESP', 'FRA', 'IRN', 'ITA', 'KOR', 'MMR', 'THA', 'TUR', 'TZA', 'UKR', 'ZAF']
country_cluster_kmeans = ['COD', 'BDI', 'BOL', 'BTN', 'CNR', 'GNQ', 'IRQ', 'LAO', 'MMR', 'MNG', 'RWA', 'UGA']
country_cluster_kmeans_Lloren = ["COD", "DEU", "EGY", "ETH", "FRA", "GBR", "IRN", "ITA", "PHL", "THA", "TUR", "VNM"]
country_cluster_kmeans_Lloren_2 = ["BFA", "CHL", "ECU", "GTM", "KAZ", "KHM", "MLI", "MWI", "NER", "NLD", "SEN", "ZMB", "ZWE"]

country_list_fdp = '''"ABW" "AFG" "AGO" "ALB" "ARE" "ARM" "ATG" "AUS" "AUT" "AZE" "BDI" "BEL" "BEN" "BFA" "BGR"
 "BHR" "BHS" "BIH" "BLR" "BLZ" "BMU" "BOL" "BRB" "BRN" "BTN" "BWA" "CAF" "CAN" "CHE" "CHL"
 "CIV" "CMR" "COM" "CPV" "CRI" "CUB" "CYP" "CZE" "DJI" "DNK" "DOM" "DZA" "ECU" "ERI" "EST"
 "FIN" "FJI" "FRO" "FSM" "GAB" "GEO" "GHA" "GIB" "GIN" "GMB" "GNB" "GNQ" "GRC" "GRD" "GRL"
 "GTM" "GUM" "GUY" "HND" "HRV" "HTI" "HUN" "IMN" "IRL" "IRQ" "ISL" "ISR" "JAM" "JOR" "KAZ"
 "KGZ" "KHM" "KIR" "KWT" "LAO" "LBN" "LBR" "LBY" "LCA" "LIE" "LKA" "LSO" "LTU" "LUX" "LVA"
 "MAF" "MAR" "MDA" "MDG" "MDV" "MKD" "MLI" "MLT" "MNE" "MNG" "MOZ" "MRT" "MUS" "MWI" "MYS"
"NAM" "NCL" "NER" "NIC" "NLD" "NOR" "NPL" "NZL" "OMN" "PAN" "PER" "PNG" "POL" "PRI" "PRK"
"PRT" "PRY" "PSE" "PYF" "QAT" "ROU" "RWA" "SAU" "SDN" "SEN" "SGP" "SLB" "SLE" "SLV" "SOM"
"SRB" "SSD" "SUR" "SVK" "SVN" "SWE" "SWZ" "SYC" "SYR" "TCA" "TCD" "TGO" "TJK" "TKM" "TLS"
"TON" "TTO" "TUN" "TUV" "UGA" "URY" "VCT" "VEN" "VGB" "VIR" "VUT" "WSM" "ZMB" "ZWE"'''
#remove the character \n 
country_list_fdp = country_list_fdp.replace('\n', '')

country_list_fdp = country_list_fdp.split(' ')
country_list_fdp = [str(country) for country in country_list_fdp]
country_list_fdp = [country.replace('"', '') for country in country_list_fdp]


In [613]:
all_df

""


In [614]:
all_df

""


In [610]:
all_df

""


In [542]:
#all_df = all_df[all_df['country'].isin(country_cluster_kmeans_Lloren)]
all_df.drop(['country', 'year'], axis=1, inplace=True)

X = all_df.drop('life_exp', axis=1)
y = all_df['life_exp']

regressor = Regression(predictor = 'all')
regressor.fit(X, y)


          
██╗░░░░░██╗░░░██╗░█████╗░██╗███████╗███████╗██████╗░░░░░░░███╗░░░███╗██╗░░░░░
██║░░░░░██║░░░██║██╔══██╗██║██╔════╝██╔════╝██╔══██╗░░░░░░████╗░████║██║░░░░░
██║░░░░░██║░░░██║██║░░╚═╝██║█████╗░░█████╗░░██████╔╝█████╗██╔████╔██║██║░░░░░
██║░░░░░██║░░░██║██║░░██╗██║██╔══╝░░██╔══╝░░██╔══██╗╚════╝██║╚██╔╝██║██║░░░░░
███████╗╚██████╔╝╚█████╔╝██║██║░░░░░███████╗██║░░██║░░░░░░██║░╚═╝░██║███████╗
╚══════╝░╚═════╝░░╚════╝░╚═╝╚═╝░░░░░╚══════╝╚═╝░░╚═╝░░░░░░╚═╝░░░░░╚═╝╚══════╝
 

Started LuciferML [ ✓ ]

Preprocessing Started [*]

Preprocessing Done [ ✓ ]

Training LuciferML [*]



Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.


Training Done [ ✓ ]

Results Below



Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.


,Name,R2 Score,Mean Absolute Error,Root Mean Squared Error,KFold Accuracy,Model
0,Linear Regression,7.69,6.78,8.37,6.44,LinearRegression()
1,Stochastic Gradient Descent Regressor,7.60,6.80,8.38,6.48,SGDRegressor()
2,Kernel Ridge Regressor,-6609.15,70.88,71.37,-6678.03,KernelRidge()
3,Elastic Net Regressor,4.42,6.90,8.52,3.80,ElasticNet()
4,Bayesian Ridge Regressor,7.69,6.78,8.37,6.44,BayesianRidge()
5,Support Vector Regressor,3.54,6.43,8.56,2.26,SVR()
6,K-Neighbors Regressor,19.03,5.67,7.84,23.80,KNeighborsRegressor()
7,Decision Trees Regressor,27.47,4.05,7.42,14.82,DecisionTreeRegressor()
8,Random Forest Regressor,49.11,3.95,6.22,51.32,"(DecisionTreeRegressor(max_features=1.0, rando..."
9,Gradient Boost Regressor,35.90,5.35,6.98,37.57,([DecisionTreeRegressor(criterion='friedman_ms...



Completed LuciferML Run [ ✓ ]

Saved Best Model to lucifer_ml_info/best/regression/models/Random_Forest_Regressor_1675530862.pkl and its scaler to lucifer_ml_info/best/regression/scalers/Random_Forest_Regressor_Scaler_1675530862.pkl 

Time Elapsed :  42.15 seconds 



In [554]:
# laod a csv in a dataframe from src\data\fake_for_prediction_COD_2018.csv
df = pd.read_csv(os.path.join(cwd, 'data', 'fake_for_prediction_COD_2018.csv'), sep=',', header=0)
df_city = df.drop(['country', 'iso_alpha3', 'name_1'], axis=1)
df_region = df.drop(['country', 'iso_alpha3', 'name_2'], axis=1)


In [555]:
#update the life_exp_pred_city column a 20%
df_city['pm_2018'] = df_city['pm_2018'] * 0.8
df_region['pm_2018'] = df_region['pm_2018'] * 0.8

In [556]:
df_city_names = df['name_1'].values
df_region_names = df['name_2'].values

np_city = np.array(df_city[['population', 'pm_2018']])
np_region = np.array(df_region[['population', 'pm_2018']])

life_exp_pred_city = regressor.predict(np_city)
life_exp_pred_region = regressor.predict(np_region)

X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names


In [557]:
#append the prediction to the dataframe
df_city['life_exp_pred_city'] = life_exp_pred_city
df_region['life_exp_pred_region'] = life_exp_pred_region

In [558]:
df_city.to_csv(os.path.join(cwd, 'RHINOS', 'cities_20_less.csv') , index=False)
df_region.to_csv(os.path.join(cwd, 'RHINOS', 'regions_20_less.csv') , index=False)

In [577]:
nested_df[(nested_df['year'].astype(int) <= 2006) & (nested_df['year'].astype(int) >= 1999)]

,country,year,life_exp,population,pm
1,ABW,1999,73.56,86867.00,3.61
2,ABW,2000,73.57,89101.00,2.40
3,ABW,2001,73.65,90691.00,2.48
4,ABW,2002,73.73,91781.00,2.22
5,ABW,2003,73.75,92701.00,3.09
...,...,...,...,...,...
4120,ZWE,2002,44.56,11984644.00,15.98
4121,ZWE,2003,43.39,12075828.00,17.69
4122,ZWE,2004,44.50,12160881.00,15.33
4123,ZWE,2005,44.77,12224753.00,17.75


In [581]:
all_years.sort_values(by=['country', 'year'], inplace=True)
all_years.reset_index(drop=True)

,country,year,min,max,avg
0,AFG,1999,18.40,65.00,38.92
1,AFG,2000,20.10,67.80,37.64
2,AFG,2001,10.40,90.00,45.62
3,AFG,2002,10.00,85.80,46.91
4,AFG,2003,15.50,89.20,47.35
...,...,...,...,...,...
1507,ZWE,2002,7.30,13.90,9.86
1508,ZWE,2003,6.90,12.00,8.72
1509,ZWE,2004,6.20,10.70,7.64
1510,ZWE,2005,7.20,13.20,8.97


In [579]:
all_years

,country,year,min,max,avg
57,AFG,1999,18.40,65.00,38.92
57,AFG,2000,20.10,67.80,37.64
57,AFG,2001,10.40,90.00,45.62
57,AFG,2002,10.00,85.80,46.91
57,AFG,2003,15.50,89.20,47.35
...,...,...,...,...,...
173,ZWE,2002,7.30,13.90,9.86
173,ZWE,2003,6.90,12.00,8.72
173,ZWE,2004,6.20,10.70,7.64
173,ZWE,2005,7.20,13.20,8.97


In [433]:
df_life_exp = all_df.pivot_table(index='country', columns='year', values='life_exp')
#rename the columns with year_life_exp
df_life_exp.columns = [str(col) + '_life_exp' for col in df_life_exp.columns]

df_population = all_df.pivot_table(index='country', columns='year', values='population')
#rename the columns with year_population
df_population.columns = [str(col) + '_population' for col in df_population.columns]

df_pm = all_df.pivot_table(index='country', columns='year', values='pm')
#rename the columns with year_pm
df_pm.columns = [str(col) + '_pm' for col in df_pm.columns]

df = pd.concat([df_life_exp, df_population, df_pm], axis=1)
df.reset_index(inplace=True)


X does not have valid feature names, but StandardScaler was fitted with feature names


array([70.34007643])

In [367]:
df_life_exp = all_df.pivot_table(index='country', columns='year', values='life_exp')
#rename the columns with year_life_exp
df_life_exp.columns = [str(col) + '_life_exp' for col in df_life_exp.columns]

df_population = all_df.pivot_table(index='country', columns='year', values='population')
#rename the columns with year_population
df_population.columns = [str(col) + '_population' for col in df_population.columns]

df_pm = all_df.pivot_table(index='country', columns='year', values='pm')
#rename the columns with year_pm
df_pm.columns = [str(col) + '_pm' for col in df_pm.columns]

df = pd.concat([df_life_exp, df_population, df_pm], axis=1)
df.reset_index(inplace=True)


In [287]:
array_new = np.zeros([len(nested_df['country'].unique()), 21, 4])
for a, i in enumerate(nested_df['country'].unique()):
    array_new[a] = np.array(nested_df[nested_df['country'] == i][['year', 'life_exp', 'population', 'pm']])

ValueError: could not broadcast input array from shape (19,4) into shape (21,4)

,country,year,life_exp,population,pm
462,BMU,2000,77.89,61833.00,4.37
463,BMU,2001,77.89,62504.00,3.97
464,BMU,2002,78.09,62912.00,3.73
465,BMU,2003,78.33,63325.00,3.72
466,BMU,2004,78.49,63740.00,3.78
467,BMU,2005,78.69,64154.00,4.06
468,BMU,2006,78.89,64523.00,3.44
469,BMU,2007,78.93,64888.00,3.97
470,BMU,2008,79.24,65273.00,3.22
471,BMU,2009,79.39,65636.00,2.71


In [279]:
np.array(df[df['country'] == i].drop('country', axis=1))

array([['1998', 2.5],
       ['1999', 3.6099999],
       ['2000', 2.4000001],
       ['2001', 2.48],
       ['2002', 2.22],
       ['2003', 3.0899999],
       ['2004', 2.99],
       ['2005', 3.48],
       ['2006', 2.79],
       ['2007', 2.99],
       ['2008', 3.3900001],
       ['2009', 2.4200001],
       ['2010', 4.8200002],
       ['2011', 3.27],
       ['2012', 2.98],
       ['2013', 2.5899999],
       ['2014', 2.1600001],
       ['2015', 2.26],
       ['2016', 2.99],
       ['2017', 2.99],
       ['2018', 2.4000001]], dtype=object)

In [270]:
len(df['country'].unique())

236

In [200]:

    
nested_df.to_csv(os.path.join(cwd, 'RHINOS', 'data', 'fake_df.csv'), index=False)